In [4]:
# pip install pandas numpy matplotlib seaborn scikit-learn jupyter

In [1]:
# **DATASET PRE-PROCESSING**
import os
import numpy as np
import pandas as pd

In [2]:
# changed sessions selection from 2005 - 2023 to link with the happiness dataset.
sessions = np.arange(60, 79)
data=[]

for session in sessions:
    directory = "../data/TXT/Session "+str(session)+" - "+str(1945+session)
    # directory = f"./TXT/Session {session} - {1945+session}"
    for filename in os.listdir(directory):
        # f = open(os.path.join(directory, filename))
     
        #if filename.index("_") == 1: continue
        #print(filename)
        with open(os.path.join(directory, filename), encoding='utf-8') as f:
            if filename[0]==".": #ignore hidden files
                continue
            splt = filename.split("_")
            data.append([session, 1945+session, splt[0], f.read()])

df_speech = pd.DataFrame(data, columns=['Session','Year','ISO-alpha3 Code','Speech'])

df_speech.tail()

,Session,Year,ISO-alpha3 Code,Speech
3660,78,2023,TCD,It is a pleasure and an honour for me to take ...
3661,78,2023,LVA,First of all. I would like to congratulate Mr....
3662,78,2023,ARE,I would like to sincerely congratulate the Pre...
3663,78,2023,BLZ,It is with immense pride that Belize offers it...
3664,78,2023,LSO,"For the past 78 years, the world’s leaders hav..."


In [3]:
df_speech.shape
# 4 columns with 3664 rows

df_speech.info()
# from the 'Non-Null Count' column it's seems like we have no missing data

missing_data = df_speech.isnull()
print(missing_data)

missing_sum = missing_data.sum()
print(missing_sum)
# no missing values to be dealt with

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3665 entries, 0 to 3664
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Session          3665 non-null   int64 
 1   Year             3665 non-null   int64 
 2   ISO-alpha3 Code  3665 non-null   object
 3   Speech           3665 non-null   object
dtypes: int64(2), object(2)
memory usage: 114.7+ KB
      Session   Year  ISO-alpha3 Code  Speech
0       False  False            False   False
1       False  False            False   False
2       False  False            False   False
3       False  False            False   False
4       False  False            False   False
...       ...    ...              ...     ...
3660    False  False            False   False
3661    False  False            False   False
3662    False  False            False   False
3663    False  False            False   False
3664    False  False            False   False

[3665 rows x 4 colu

In [4]:
df_codes = pd.read_csv('../data/UNSD — Methodology.csv', delimiter=';')
df_codes.head()

,Global Code,Global Name,Region Code,Region Name,Sub-region Code,Sub-region Name,Intermediate Region Code,Intermediate Region Name,Country or Area,M49 Code,ISO-alpha2 Code,ISO-alpha3 Code,Least Developed Countries (LDC),Land Locked Developing Countries (LLDC),Small Island Developing States (SIDS)
0,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Algeria,12,DZ,DZA,NaN,NaN,NaN
1,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Egypt,818,EG,EGY,NaN,NaN,NaN
2,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Libya,434,LY,LBY,NaN,NaN,NaN
3,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Morocco,504,MA,MAR,NaN,NaN,NaN
4,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Sudan,729,SD,SDN,x,NaN,NaN


In [5]:
# merge the two dataframes on 'ISO-alpha3 Code'
merged_df = pd.merge(df_codes, df_speech, on="ISO-alpha3 Code")
merged_df = merged_df[["Country or Area", "Region Name","Sub-region Name", "ISO-alpha3 Code", "Session", "Year", "Speech"]]

# Set the index to a multindex
merged_df = merged_df.set_index(['Country or Area','Year'])

# Rename the index columns
merged_df = merged_df.rename_axis(['Country name', 'year'])


merged_df.head()

Region Name  Sub-region Name ISO-alpha3 Code  Session  \
Country name year                                                         
Algeria      2005      Africa  Northern Africa             DZA       60   
             2006      Africa  Northern Africa             DZA       61   
             2007      Africa  Northern Africa             DZA       62   
             2008      Africa  Northern Africa             DZA       63   
             2009      Africa  Northern Africa             DZA       64   

                                                              Speech  
Country name year                                                     
Algeria      2005  At\nthis crucial moment, when it is debating q...  
             2006  At a \ntime when the United Nations continues ...  
             2007  I have  the particular pleasure of conveying t...  
             2008  I wish \nat the outset to congratulate Mr. Mig...  
             2009  Allow \nme first to express the pleasure of th...

In [6]:
happinessdataframe = pd.read_excel('../data/DataForTable2.1.xls', index_col=[0,1]) 

print(happinessdataframe.shape)
# 9 columns with 2363 rows

happinessdataframe.info()
# from the 'Non-Null Count' column it's seems like Life ladder is the only column with no missing values

missing_data = happinessdataframe.isnull()
missing_sum = missing_data.sum()
print(missing_sum)
# There are some missing values in this dataset

happinessdataframe.head()

(2363, 9)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2363 entries, ('Afghanistan', np.int64(2008)) to ('Zimbabwe', np.int64(2023))
Data columns (total 9 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Life Ladder                       2363 non-null   float64
 1   Log GDP per capita                2335 non-null   float64
 2   Social support                    2350 non-null   float64
 3   Healthy life expectancy at birth  2300 non-null   float64
 4   Freedom to make life choices      2327 non-null   float64
 5   Generosity                        2282 non-null   float64
 6   Perceptions of corruption         2238 non-null   float64
 7   Positive affect                   2339 non-null   float64
 8   Negative affect                   2347 non-null   float64
dtypes: float64(9)
memory usage: 179.2+ KB
Life Ladder                           0
Log GDP per capita                   28
Social support

Life Ladder  Log GDP per capita  Social support  \
Country name year                                                    
Afghanistan  2008     3.723590            7.350416        0.450662   
             2009     4.401778            7.508646        0.552308   
             2010     4.758381            7.613900        0.539075   
             2011     3.831719            7.581259        0.521104   
             2012     3.782938            7.660506        0.520637   

                   Healthy life expectancy at birth  \
Country name year                                     
Afghanistan  2008                         50.500000   
             2009                         50.799999   
             2010                         51.099998   
             2011                         51.400002   
             2012                         51.700001   

                   Freedom to make life choices  Generosity  \
Country name year                                             
Afghanistan  2008                      0.718114    0.164055   
             2009                      0.678896    0.187297   
             2010                      0.600127    0.117861   
             2011                      0.495901    0.160098   
             2012                      0.530935    0.234157   

                   Perceptions of corruption  Positive affect  Negative affect  
Country name year                                                               
Afghanistan  2008                   0.881686         0.414297         0.258195  
             2009                   0.850035         0.481421         0.237092  
             2010                   0.706766         0.516907         0.275324  
             2011                   0.731109         0.479835         0.267175  
             2012                   0.775620         0.613513         0.267919

In [7]:
# My goal is to iterate through each country in this dataframe and replace any missing values with the mean of that column
# for that particular country

# This shows the mean data for each column for the country of Afghanistan
happinessdataframe.loc["Afghanistan"].mean()

Life Ladder                          3.219917
Log GDP per capita                   7.585624
Social support                       0.476654
Healthy life expectancy at birth    52.711667
Freedom to make life choices         0.480475
Generosity                           0.056325
Perceptions of corruption            0.835824
Positive affect                      0.421804
Negative affect                      0.370840
dtype: float64

In [8]:
# This line of code fills any missing value in a column with the mean value for that particular country
happinessdataframe.loc["Afghanistan"].fillna(happinessdataframe.loc["Afghanistan"].mean())

,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
year,,,,,,,,,
2008,3.723590,7.350416,0.450662,50.500000,0.718114,0.164055,0.881686,0.414297,0.258195
2009,4.401778,7.508646,0.552308,50.799999,0.678896,0.187297,0.850035,0.481421,0.237092
2010,4.758381,7.613900,0.539075,51.099998,0.600127,0.117861,0.706766,0.516907,0.275324
2011,3.831719,7.581259,0.521104,51.400002,0.495901,0.160098,0.731109,0.479835,0.267175
2012,3.782938,7.660506,0.520637,51.700001,0.530935,0.234157,0.775620,0.613513,0.267919
2013,3.572100,7.680333,0.483552,52.000000,0.577955,0.059246,0.823204,0.547417,0.273328
2014,3.130896,7.670638,0.525568,52.299999,0.508514,0.102330,0.871242,0.491641,0.374861
2015,3.982855,7.653833,0.528597,52.599998,0.388928,0.078218,0.880638,0.491410,0.339276
2016,4.220169,7.650370,0.559072,52.924999,0.522566,0.040480,0.793246,0.501409,0.348332


In [9]:
# Group by the first level of the MultiIndex (Country name)
# apply(lambda group: group.fillna(group.mean())): For each group (which corresponds to each country), 
# it applies a function that fills the NaN values with the column-wise mean for that specific group.
happinessdataframe = happinessdataframe.groupby(level=0, group_keys=False).apply(lambda group: group.fillna(group.mean()))
happinessdataframe.info()

missing_data = happinessdataframe.isnull()
missing_sum = missing_data.sum()
print(missing_sum)
# We haven't filled in all of the missing data and this is because some countries like China
# Do not have any data for their column 'Perception of corruption' which means we can't calculate a mean for that group.
# This is fine for now but if it needs to be changed we can come back later.

# exported the new dataframe so I could have a look at the full set in excel
#happinessdataframe.to_excel("output.xlsx")

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2363 entries, ('Afghanistan', np.int64(2008)) to ('Zimbabwe', np.int64(2023))
Data columns (total 9 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Life Ladder                       2363 non-null   float64
 1   Log GDP per capita                2354 non-null   float64
 2   Social support                    2362 non-null   float64
 3   Healthy life expectancy at birth  2313 non-null   float64
 4   Freedom to make life choices      2363 non-null   float64
 5   Generosity                        2354 non-null   float64
 6   Perceptions of corruption         2333 non-null   float64
 7   Positive affect                   2361 non-null   float64
 8   Negative affect                   2362 non-null   float64
dtypes: float64(9)
memory usage: 179.2+ KB
Life Ladder                          0
Log GDP per capita                   9
Social support            

In [10]:
# Perform the merge based on Country and Year from both dataframes
final_df = pd.merge(merged_df, happinessdataframe, on=["Country name","year"], suffixes=('_left', '_right'))

# exported the file just so I could double check everything has merged properly
# which it looks like it has
#final_df.to_excel("output.xlsx")

In [11]:
# Now if we take a look at the new dataframe
# TODO -> Future work, see which countries where dropped.
final_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2038 entries, ('Algeria', np.int64(2010)) to ('New Zealand', np.int64(2023))
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Region Name                       2038 non-null   object 
 1   Sub-region Name                   2038 non-null   object 
 2   ISO-alpha3 Code                   2038 non-null   object 
 3   Session                           2038 non-null   int64  
 4   Speech                            2038 non-null   object 
 5   Life Ladder                       2038 non-null   float64
 6   Log GDP per capita                2033 non-null   float64
 7   Social support                    2037 non-null   float64
 8   Healthy life expectancy at birth  2022 non-null   float64
 9   Freedom to make life choices      2038 non-null   float64
 10  Generosity                        2033 non-null   float64
 11  Perceptions of c

In [12]:
final_df.head()

Region Name  Sub-region Name ISO-alpha3 Code  Session  \
Country name year                                                         
Algeria      2010      Africa  Northern Africa             DZA       65   
             2011      Africa  Northern Africa             DZA       66   
             2012      Africa  Northern Africa             DZA       67   
             2014      Africa  Northern Africa             DZA       69   
             2016      Africa  Northern Africa             DZA       71   

                                                              Speech  \
Country name year                                                      
Algeria      2010  First of \nall, I would like to congratulate y...   
             2011  His \nExcellency Mr. Abdelaziz Bouteflika, Pre...   
             2012  ﻿I would\nlike to take this opportunity to war...   
             2014  I should like at the outset to express my sati...   
             2016  I am honoured and privileged to address the As...   

                   Life Ladder  Log GDP per capita  Social support  \
Country name year                                                    
Algeria      2010     5.463567            9.306355        0.811683   
             2011     5.317194            9.315958        0.810234   
             2012     5.604596            9.329962        0.839397   
             2014     6.354898            9.355415        0.818189   
             2016     5.340854            9.383312        0.748588   

                   Healthy life expectancy at birth  \
Country name year                                     
Algeria      2010                         65.500000   
             2011                         65.599998   
             2012                         65.699997   
             2014                         65.900002   
             2016                         66.099998   

                   Freedom to make life choices  Generosity  \
Country name year                                             
Algeria      2010                      0.592696   -0.212297   
             2011                      0.529561   -0.187624   
             2012                      0.586663   -0.179103   
             2014                      0.520711   -0.132543   
             2016                      0.520711   -0.132543   

                   Perceptions of corruption  Positive affect  Negative affect  
Country name year                                                               
Algeria      2010                   0.618038         0.540448         0.266245  
             2011                   0.637982         0.502736         0.254897  
             2012                   0.690116         0.540059         0.229716  
             2014                   0.688052         0.558359         0.176866  
             2016                   0.688052         0.565026         0.377112

In [13]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/christinaperdue/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/christinaperdue/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/christinaperdue/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/christinaperdue/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [20]:
from nltk.probability import FreqDist
from nltk import word_tokenize
from nltk.corpus import stopwords

def preprocess(words):
    sw = stopwords.words("english")
    no_sw = []
    for w in words:
        if (w.lower() not in sw) & (w.isalpha()):
            no_sw.append(w.lower())
    return no_sw

text = final_df.loc["Germany",2022]["Speech"]

words = word_tokenize(text)
words = preprocess(words)
freq = FreqDist(words)

# lets create a .csv file in data 
target_words_education = ["equitable", "pedagogy", "knowledge", "learning", "learn", "knowledges", "equity", 
                          "development", "teaching", "teacher", "education", "literacy", "numeracy", "vocational", 
                          "technical", "accessible", "primary", "secondary", "tertiary", "degree","degrees", 
                          "university", "college", "school", "phd", "stem", "science", "arts", "technology", "tech"]
target_words_female = ["gender", "women", "woman", "girl", "girls", "female", "non-binary", "sexes", "equality"]


In [21]:
# Assign a variable all of the information from 2022
tt2_df = final_df.loc(axis=0)[:,2022]

temp_dic = {} # this is used in the final for loop

# This function takes in the words of a speech and compares it with our target words
def count_unique_matches(words, target_words):
    
    target_set = set(target_words) # Convert to set for faster look ups
    
    matched_words = target_set.intersection(words) # This collects what target rooms are present in the speech

    return len(matched_words) # return the frequency of the matched words in the speech

# iterate through the 2022 data by row
for index, row in tt2_df.iterrows():
    
    # set our key of our dictionary to the country name and let that equal to the frequency of matched words
    temp_dic[index[0]] =  count_unique_matches(word_tokenize(row["Speech"]), target_words_education)
    
print(temp_dic)

{'Algeria': 2, 'Egypt': 3, 'Libya': 2, 'Morocco': 2, 'Tunisia': 3, 'Comoros': 2, 'Ethiopia': 5, 'Kenya': 7, 'Madagascar': 6, 'Malawi': 2, 'Mauritius': 6, 'Mozambique': 5, 'Uganda': 5, 'Zambia': 5, 'Zimbabwe': 8, 'Cameroon': 1, 'Chad': 1, 'Gabon': 1, 'Botswana': 5, 'Eswatini': 3, 'Lesotho': 1, 'Namibia': 6, 'South Africa': 7, 'Benin': 3, 'Gambia': 1, 'Ghana': 2, 'Guinea': 2, 'Liberia': 2, 'Mali': 4, 'Mauritania': 2, 'Niger': 6, 'Nigeria': 2, 'Senegal': 2, 'Sierra Leone': 14, 'Togo': 6, 'Dominican Republic': 4, 'Jamaica': 4, 'Costa Rica': 3, 'El Salvador': 1, 'Guatemala': 2, 'Honduras': 2, 'Mexico': 2, 'Nicaragua': 2, 'Panama': 7, 'Argentina': 4, 'Brazil': 3, 'Chile': 5, 'Colombia': 1, 'Ecuador': 3, 'Paraguay': 1, 'Peru': 5, 'Uruguay': 2, 'Canada': 4, 'Kazakhstan': 3, 'Kyrgyzstan': 2, 'Tajikistan': 3, 'Uzbekistan': 3, 'Japan': 2, 'Mongolia': 5, 'Cambodia': 2, 'Indonesia': 1, 'Malaysia': 2, 'Philippines': 5, 'Singapore': 2, 'Thailand': 5, 'Bangladesh': 6, 'India': 5, 'Nepal': 6, 'Pakistan

In [22]:
# This could be useful for eda in describing what words show up in each text for a country
target_set = set(target_words_education)
print(target_set.intersection(words))

{'development', 'education'}


- Remove everything related to our target words in all the speeches from 2022
- potentially we want to look for phrases rather than just words in the speech
- Instead of using a dictionary we use a data frame to store the Year, Country name and frequency of target words or just append the frequencies to the original dataset.
